In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
dataset_url = 'https://www.kaggle.com/bingazuran/nsfwsfwdataset2'

In [4]:
!pip install jovian opendatasets --upgrade --quiet
import opendatasets as od
od.download(dataset_url)

     |████████████████████████████████| 68 kB 3.1 MB/s 
Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: bingazuran
Your Kaggle Key: ··········


100%|██████████| 1.12G/1.12G [00:06<00:00, 195MB/s]


In [5]:
import os
import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [6]:
data_dir = '/content/nsfwsfwdataset2/NSFW-FSW-Dataset-Omar-3'

print(os.listdir(data_dir))
classes = os.listdir(data_dir + "/train")
print(classes)

['train', 'test']
['sfw', 'nsfw']


In [7]:
#import os

#file_path = './nsfwsfwdataset/bug-omar-dataset/train/sfw/4bb193d3-edfa-4e0d-9ff6-36fc859de90a.jpg'
#os.remove(file_path)

In [8]:
transform_train = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomResizedCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

transform_test = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomResizedCrop(224), 
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

In [9]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor

In [11]:
trainset = ImageFolder(data_dir+'/train', transform=transform_train)
testset = ImageFolder(data_dir+'/test', transform=transform_test)

In [12]:
trainset

Dataset ImageFolder
    Number of datapoints: 5000
    Root location: /content/nsfwsfwdataset2/NSFW-FSW-Dataset-Omar-3/train
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=bilinear, max_size=None, antialias=None)
               RandomResizedCrop(size=(224, 224), scale=(0.08, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [13]:
batch_size = 16
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

In [14]:
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

In [15]:
img, label = trainset[0]
print(img.shape, label)
img

torch.Size([3, 224, 224]) 0


tensor([[[-0.3961, -0.4431, -0.4667,  ...,  0.7961,  0.8196,  0.8353],
         [-0.3569, -0.4353, -0.4745,  ...,  0.8039,  0.8118,  0.8196],
         [-0.3333, -0.4275, -0.4745,  ...,  0.7882,  0.7882,  0.7961],
         ...,
         [ 0.1765,  0.1451,  0.1608,  ...,  0.1216,  0.1294,  0.0980],
         [ 0.1686,  0.1451,  0.1608,  ...,  0.1137,  0.1216,  0.1294],
         [ 0.1765,  0.1608,  0.1765,  ...,  0.1059,  0.1059,  0.1137]],

        [[-0.6941, -0.7255, -0.7412,  ...,  0.2078,  0.2235,  0.2392],
         [-0.6706, -0.7255, -0.7490,  ...,  0.2157,  0.2235,  0.2314],
         [-0.6549, -0.7176, -0.7569,  ...,  0.2000,  0.2078,  0.2157],
         ...,
         [-0.3804, -0.4118, -0.3961,  ..., -0.1451, -0.1373, -0.1529],
         [-0.3961, -0.4275, -0.4118,  ..., -0.1373, -0.1373, -0.1294],
         [-0.3961, -0.4118, -0.3961,  ..., -0.1451, -0.1529, -0.1373]],

        [[-0.8353, -0.8667, -0.8510,  ..., -0.0275,  0.0039,  0.0275],
         [-0.8196, -0.8588, -0.8588,  ..., -0

In [16]:
from torchsummary import summary

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
from torchvision import models
batch_size = 16

In [18]:
vgg = models.vgg16_bn(pretrained=True)

Downloading: "https://download.pytorch.org/models/vgg16_bn-6c64b313.pth" to /root/.cache/torch/hub/checkpoints/vgg16_bn-6c64b313.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [19]:
for param in vgg.parameters():
    param.requires_grad = False

In [20]:
num_classes = 2
final_in_features = vgg.classifier[6].in_features
vgg.classifier[6] = nn.Linear(final_in_features, num_classes)

In [21]:
for param in vgg.parameters():
    if param.requires_grad:
        print(param.shape)

torch.Size([2, 4096])
torch.Size([2])


In [22]:
vgg = vgg.to(device)
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adagrad(vgg.parameters(), lr=0.05)

In [23]:
def evaluation(dataloader, model):
    total, correct = 0, 0
    for data in dataloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, pred = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (pred == labels).sum().item()
    return 100 * correct / total

In [24]:
loss_epoch_arr = []
max_epochs = 5
train_accuracy = []
test_accuracy = []

n_iters = np.ceil(5000/batch_size)

for epoch in range(max_epochs):

    for i, data in enumerate(trainloader, 0):

        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        opt.zero_grad()

        outputs = vgg(inputs)
        loss = loss_fn(outputs, labels)
        loss.backward()
        opt.step()
        
        if i % 100 == 0:
            print('Iteration: %d/%d, Loss: %0.2f' % (i, n_iters, loss.item()))
            
        del inputs, labels, outputs
        torch.cuda.empty_cache()
        
    loss_epoch_arr.append(loss.item())

    train_a = evaluation(trainloader, vgg)
    test_a = evaluation(testloader, vgg)
        
    print('Epoch: %d/%d, Test acc: %0.2f, Train acc: %0.2f' % (
        epoch, max_epochs, test_a, train_a))
    train_accuracy.append(train_a)
    test_accuracy.append(test_a)
    
    
plt.plot(loss_epoch_arr)
plt.show()

Iteration: 0/313, Loss: 0.64
Iteration: 100/313, Loss: 2.22
Iteration: 200/313, Loss: 0.90
Iteration: 300/313, Loss: 0.70
Epoch: 0/5, Test acc: 86.00, Train acc: 86.48
Iteration: 0/313, Loss: 2.40
Iteration: 100/313, Loss: 0.76


KeyboardInterrupt: ignored

In [ ]:
plt.plot(loss_iterations)
plt.ylabel('loss')
plt.title("LOSS V/S ITERATIONS")
plt.legend()
plt.show()